In [3]:
import os

from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

In [4]:
ROOT_DIR = '../'
MODEL_PATH = ROOT_DIR + 'models/Patryk-ResNeXt-more-layers-model.pkt'
TRAINING_LABELS_PATH = ROOT_DIR + 'data/training_labels.csv'
TEST_DATA_PATH = ROOT_DIR + 'data/live_test_images'
SUBMISSION_FILE = ROOT_DIR + 'results/submission.csv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cpu


/home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
class TestDataset(Dataset):

    def __init__(self, img_dir, transform):
        self.transform = transform
        self.images = [os.path.join(img_dir, fname) for fname in os.listdir(img_dir)]

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)
        return image
  
    def __len__(self):
        return len(self.images)

In [6]:
model = torch.load(MODEL_PATH, map_location=DEVICE)

In [7]:
batch_size = 16
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_set = TestDataset(TEST_DATA_PATH, transform)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

In [9]:
results = []

for batch in test_loader:
    res = model(batch.to(DEVICE))
    res = torch.sigmoid(res).to(torch.float32).cpu().detach().numpy()
    results.append(res)

In [10]:
optimal_thresholds = [
    0.21428571, 0.45454545, 0.21875   , 0.41743119, 0.35714286,
       0.44444444, 0.4       , 0.359375  , 0.38613861, 0.24      ,
       0.33333333, 0.30813953, 0.35443038, 0.35714286, 0.34674923,
       0.33333333, 0.4       , 0.34722222, 0.35      , 0.375     ,
       0.28985507, 0.33333333, 0.28205128, 0.23076923, 0.15789474,
       0.38505747, 0.33793103, 0.37837838, 0.35      , 0.3021978 ,
       0.36170213, 0.46153846, 0.38888889, 0.31818182, 0.15909091,
       0.42672414, 0.5       , 0.36824324]

In [15]:
df = pd.read_csv(TRAINING_LABELS_PATH)
df_test = pd.DataFrame(np.vstack(results), columns = df.columns[1:])
df_test = (df_test > optimal_thresholds).astype(int) # Threshold optimization
names = pd.DataFrame({'Name': [Path(el).parts[-1] for el in test_loader.dataset.images]})

submission = pd.concat([names, df_test], axis = 1)
submission.to_csv(SUBMISSION_FILE, index = False)